In [35]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import requests
import os
import pandas as pd
from datetime import datetime, timedelta
import dateparser 
import re

base_url  = 'https://www.indiatvnews.com/'
country = 'bangladesh'

initial_url = f'{base_url}/topic/{country}'

#FINDING NUMBER OF PAGES

# page = 1
# while True:
    
        
#     initial_url = f'{base_url}/topic/{country}/{page}'

#     response = requests.get(initial_url)

#     soup = BeautifulSoup(response.text, 'html.parser')
#     buttons_div = soup.find('div', class_ = 'paginb clearfix')
#     invalid_next_button = buttons_div.find('li', class_ = 'nextpb dis')
    
#     if invalid_next_button:
#         break
#     else:
#         page +=1
        
    
# print(page)

#Extracting all the URLs

links = []
for i in range(1,4):
    
    initial_url = f'{base_url}/topic/{country}/{i}'
    
    response = requests.get(initial_url)
     
    soup = BeautifulSoup(response.text, 'html.parser')

    news_urls = soup.find_all('li', class_ = 'eventTracking')

    for news in news_urls:
        temp = news.find('a', href = True)
        links.append(temp['href'])


counter = 0
data_list = []

for link in links[:20]:
    
    country = 'bangladesh'
    
    if f'{country}' in link:
        
        response = requests.get(link)

        soup = BeautifulSoup(response.text, 'html.parser')

        title_tag = soup.find('h1', class_ = 'arttitle')
        title = title_tag.text if title_tag else 'Title Not Found'
        
        author_tag = soup.find('span', class_ = 'author-name')
        
        author = author_tag.find('a').text if author_tag else 'Author not found'

        date_data = soup.find('span', class_ = 'published-on')

        if date_data:

            date_ = date_data.text
            
            x = date_data.text

            x = x.split(': ',1)[1].split(' IST')[0]
            
            source_localtime = datetime.strptime(x, "%B %d, %Y %H:%M")
            bangladesh_localtime = source_localtime + timedelta(minutes=30)

        else:
            date_data = 'Date Data Not Found'

        content_div = soup.find('div', {'id':'content'})

        content = []

        if content_div:

            for useful_div in content_div.contents:
                if useful_div.name == 'p' or useful_div.name == 'h3':

                    content.append(useful_div.text)
        else:
            content.append('Content Not Found')


        temp_content = ' '.join(content)
        
        temp_content = re.sub(r'\n|\xa0', '', temp_content)
        

        temp_cuts = temp_content.split('ALSO READ', 1)
        full_content = temp_cuts[0].strip()

        content_summary_tag = soup.find('h2', class_ = 'artdec')
        content_summary = content_summary_tag.text if content_summary_tag else 'Content summary not found'
        content_summary = re.sub(r'\n|\xa0', '', content_summary)

        title_translation = 'None'
        summary_translation = 'None'
        content_translation = 'None'
    
    
        data_dict = {
            "url": link,
            "title": title,
            "content": full_content,
            "content_summary": content_summary,
            "title_translation":title_translation,
            "content_translation":content_translation,
            "summary translation":summary_translation,
            "author": author,
            "country": country,
            'source_localtime': source_localtime,
            'bangladesh_localtime': bangladesh_localtime

        }
        
        counter+=1


        if (date_data != "Date Data Not Found" and full_content != "Content Not Found" and content_summary != "Content summary not found"):
            if data_dict not in data_list:
                # Adding to data list
                data_list.append(data_dict)
                print(f'Link {counter} added')
        else:
            print(counter)
            print('Skipped due to missing info.')


df = pd.DataFrame(data_list)
df.head()

Link 1 added
Link 2 added
Link 3 added
Link 4 added
Link 5 added
Link 6 added
Link 7 added
Link 8 added
Link 9 added
Link 10 added
Link 11 added
12
Skipped due to missing info.
Link 13 added
Link 14 added
Link 15 added
Link 16 added
Link 17 added


,url,title,content,content_summary,title_translation,content_translation,summary translation,author,country,source_localtime,bangladesh_localtime
0,https://www.indiatvnews.com/news/world/banglad...,Sheikh Hasina sworn in as Prime Minister of Ba...,Bangladesh: Sheikh Hasina was on Thursday swor...,"The 76-year-old Sheikh Hasina, the daughter of...",None,None,None,Ajeet Kumar,bangladesh,2024-01-11 23:07:00,2024-01-11 23:37:00
1,https://www.indiatvnews.com/sports/other/bangl...,Bangladesh cricket captain Shakib Al Hasan win...,Bangladesh cricket team's captain Shakib Al Ha...,Shakib Al Hasan took a temporary break from cr...,None,None,None,Varun Malik,bangladesh,2024-01-08 07:35:00,2024-01-08 08:05:00
2,https://www.indiatvnews.com/news/world/banglad...,Bangladesh PM Sheikh Hasina elected for fifth ...,Bangladesh Prime Minister Sheikh Hasina's Awam...,Leaders of the Bangladesh Nationalist Party (B...,None,None,None,Raju Kumar,bangladesh,2024-01-07 23:51:00,2024-01-08 00:21:00
3,https://www.indiatvnews.com/news/world/banglad...,Bangladesh elections: Amid violence and Opposi...,Bangladesh elections 2024: Bangladeshis on Sun...,"According to the initial estimates, the voter ...",None,None,None,Ajeet Kumar,bangladesh,2024-01-07 18:55:00,2024-01-07 19:25:00
4,https://www.indiatvnews.com/news/world/banglad...,'We are very lucky...India is our trusted frie...,Bangladesh elections:Bangladesh Prime Minister...,Bangladesh elections: PM Hasina recounted the ...,None,None,None,Ajeet Kumar,bangladesh,2024-01-07 11:14:00,2024-01-07 11:44:00
